In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import json
import torch
import random
import tokenize
import numpy as np
from tqdm import tqdm
import multiprocessing
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss, MSELoss
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
f = open('../data/readme.jsonl', 'r')
dataset1 = json.loads(f.readline())
f.close()
f = open('../data/description.jsonl', 'r')
dataset2 = json.loads(f.readline())
f.close()
f = open('../data/labels01.jsonl', 'r')
labels01 = json.loads(f.readline())
f.close()
topic_number = len(labels01[0])

In [3]:
class arguments(object):
    def __init__(self):
        pass
args = arguments()

In [4]:
args.epochs = 25
args.batch_size = 32

args.total_length = 512
args.graph_length = 0
args.max_grad_norm = 1.0
args.learning_rate = 2e-5
args.weight_decay = 0.0
args.adam_epsilon = 1e-8

args.current_topic = 0
args.topic_number = topic_number
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.seed = 978438233

def set_seed():
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
set_seed()

In [5]:
dataset = []
for x, y, z in tqdm(zip(dataset1, dataset2, labels01), total = len(dataset1)):
    if (sum(z) != 0):
        dataset.append([x + y, z])
random.shuffle(dataset)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12272/12272 [00:00<00:00, 49887.96it/s]


In [6]:
class TextDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples
        
    def __len__(self):
        return len(self.examples)
        
    def __getitem__(self, item):
        x, labels = self.examples[item]
        return torch.tensor(x), torch.Tensor([labels[args.current_topic]])

test_data = TextDataset(dataset[int(len(dataset) * 0.75) :])
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, drop_last = False,
                             num_workers = 4, batch_size = args.batch_size)

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.dense = nn.Linear(768, 8192 * 4)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(8192 * 4, 2)
        self.loss_function = MSELoss()
        
    def forward(self, x, labels, training):
        x = self.dropout(x)
        x = self.dense(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        x = F.softmax(x, dim = 1)[:, 1]
        loss = self.loss_function(x.view(-1), labels.view(-1))
        if (training):
            return loss
        else:
            return x, loss

In [8]:
def evaluate(model, epoch_id):
    loss_sum = 0
    loss_cnt = 0
    y_trues = []
    y_preds = []
    bar = tqdm(test_dataloader, total = len(test_dataloader))
    for data in bar:
        x, y = data
        x = x.to(args.device)
        y = y.to(args.device)
        model.eval()
        with torch.no_grad():
            prob, loss = model(x, y, False)
            prob = prob.view(-1)
            loss_sum = loss_sum + loss.item() * x.size(0)
            loss_cnt = loss_cnt + x.size(0)
            y_preds.append((prob > 0.5).long().cpu().numpy())
            y_trues.append(y.long().view(-1).cpu().numpy())
    y_trues = np.concatenate(y_trues, 0)
    y_preds = np.concatenate(y_preds, 0)
    TP = sum([x == 1 and y == 1 for x, y in zip(y_trues, y_preds)])
    FP = sum([x == 0 and y == 1 for x, y in zip(y_trues, y_preds)])
    TN = sum([x == 0 and y == 0 for x, y in zip(y_trues, y_preds)])
    FN = sum([x == 1 and y == 0 for x, y in zip(y_trues, y_preds)])
    print('TP FP TN FN =', TP, FP, TN, FN)

    f1 = float(f1_score(y_trues, y_preds))
    rs = float(recall_score(y_trues, y_preds))
    ps = float(precision_score(y_trues, y_preds))
    os.system('mkdir -p result')
    print('f1:', f1)
    print('recall:', rs)
    print('precision:', ps)
    print('loss:', loss_sum / loss_cnt)
    f = open('result/context' + str(args.current_topic).zfill(2) + '-' + str(epoch_id).zfill(3) + '.txt', 'w')
    print(f1, rs, ps, loss_sum / loss_cnt, TP, FP, TN, FN, file = f)
    f.close()
    return f1

In [9]:
def get_loader():
    posi_data = []
    nega_data = []
    for x in dataset[: int(len(dataset) * 0.75)]:
        if (x[1][args.current_topic]):
            posi_data.append(x)
        else:
            nega_data.append(x)
    if (len(posi_data) < len(nega_data)):
        nega_data = random.sample(nega_data, max(1, len(posi_data)))
    train_data = TextDataset(posi_data + nega_data)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler = train_sampler, drop_last = False,
                                  num_workers = 4, batch_size = args.batch_size)
    return train_dataloader

for i in range(args.topic_number):
    args.current_topic = i
    model = Model()
    model.to(args.device)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)
    optimizer = torch.optim.Adam(model.parameters(), lr = args.learning_rate)
    for epoch_num in range(args.epochs):
        train_dataloader = get_loader()
        for t in tqdm(range(50)):
            for data in train_dataloader:
                x, y = data
                x = x.to(args.device)
                y = y.to(args.device)
                model.train()
                loss = model(x, y, True)
                if args.n_gpu > 1:
                    loss = loss.mean()
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        evaluate(model, epoch_num)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 163.02it/s]


TP FP TN FN = 286 382 2379 21
f1: 0.5866666666666667
recall: 0.9315960912052117
precision: 0.4281437125748503
loss: 0.09809116057497892


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 136.23it/s]


TP FP TN FN = 265 231 2530 42
f1: 0.6600249066002492
recall: 0.8631921824104235
precision: 0.5342741935483871
loss: 0.06628621781227875


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 151.49it/s]


...

TP FP TN FN = 171 41 2800 56
f1: 0.7790432801822322
recall: 0.7533039647577092
precision: 0.8066037735849056
loss: 0.025932105148374487


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 156.25it/s]


TP FP TN FN = 171 35 2806 56
f1: 0.789838337182448
recall: 0.7533039647577092
precision: 0.8300970873786407
loss: 0.025744718359548124


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 147.11it/s]


TP FP TN FN = 171 35 2806 56
f1: 0.789838337182448
recall: 0.7533039647577092
precision: 0.8300970873786407
loss: 0.025414891961176505


In [10]:
exit()